In [1]:
import numpy as np
import math
import matplotlib.pyplot as plt
from pdb import set_trace

In [2]:
def linear_regression(x, y):
    X_t = x.T
    X_y = X_t.dot(y) # x is a column vector so it needs to be transformed
    X_XT = X_t.dot(x)
    w = np.linalg.solve(X_XT, X_y) #(X_XT dot X_y)^-1
    return w

In [3]:
def model(x, weight, b):
    y_hat = x.dot(weight) + b
    return y_hat

In [4]:
def mean_square_error(y_hat, y):
    n = len(y)
    mse = (1/(2 * n)) * np.sum(np.square(y_hat-y))
    return mse

In [5]:
def l2(w, y, alpha = 0.01):
    n = len(y)
    wt_w = (w).dot(w.T) # (w^T)w
    reg = (1/(2 * n) * alpha) * wt_w # (alpha/2n) * ((w^T)w) 
    return reg

In [6]:
def regularized_mean_square_error(y, y_hat, reg):
    n = len(y)
    mse = mean_square_error(y_hat, y)
    return mse + reg

In [7]:
#Train/Test Split performed by randomly taking inputs and their associated labels and assigning them to either a training group or a test group
#---------------------------------------------------#----------------#
#                                                   |                |
#                  Training set                     |   Testing Set  |
#                                                   |                |
#---------------------------------------------------#----------------#
#train_perc should be a value between 0 and 1, eg. train_perc 0.8
def random_train_test_split(dataset, train_perc = 0.8):
    perc_of_dataset = dataset.shape[1] * train_perc
    numpy.random.shuffle(dataset)

    train = dataset_arrx[:perc_of_dataset,:] 
    test = dataset_arr[perc_of_dataset:,:]

    return train, test

In [8]:
#Validation/Test Split performed by randomly taking inputs and their associated labels and assigning them to either a validation group or a test group
#---------------------------------------------------#----------------#
#                                                   |                |
#                  Training set                     |   Testing Set  |
#                                                   |                |
#---------------------------------------------------#----------------#
#                                                             V
#                                                |------------#---------#
#                                                |            |         |
#                                                | Validation | Testing |
#                                                |     Set    |   Set   |
#                                                |------------#---------#
#train_perc should be a value between 0 and 1, eg. train_perc 0.5 for a 50/50 split of the test set 
def random_test_validation_split(parameter, labels, train_perc = 0.8):
    dataset_row_size, dataset_col_size = parameter.shape
    validation_set_size = int(dataset_row_size * train_perc)
    test_set_size = int(dataset_row_size - validation_set_size)

    #get random indices for both validation and test sets
    indices = np.random.permutation(parameter.shape[0])
    validation_p_index = indices[:validation_set_size]
    test_p_index = indices[test_set_size:]

    #validation and test labels
    validation_l = labels[validation_p_index]
    test_l = labels[test_p_index]

    #validation and test parameters 
    validation_p = parameter[validation_p_index]
    test_p = parameter[test_p_index]

    return validation_p, validation_l, test_p, test_l

In [9]:
#X is the input 
#y is the real result
#w is the weight 
#epsilon is the learning rate
#alpha is the regularized term
#reg_bool if you wish to use the regularized term
def gradient_descent(x, y, b, w, epsilon, reg_bool):
    X_t = x.T
    n = len(y)
    y_hat = model(x, w, b)

    #taking derivative 
    derivative_of_w = (1/n) * np.sum(X_t.dot(y_hat - y))
    derivative_of_b = (1/n) * np.sum(y_hat - y)

    # if regularized term is wanted then add that to the derivative with respect to w
    if reg_bool == True:
        derivative_of_w += ((alpha/n) * w)

    #this is gradient descent
    w_new = w - (epsilon * derivative_of_w)
    b_new = b - (epsilon * derivative_of_b)

    return w_new, b_new

In [10]:
#ripped from https://stackoverflow.com/questions/4601373/better-way-to-shuffle-two-numpy-arrays-in-unison
def unison_shuffled_copies(a, b):
    assert len(a) == len(b)
    p = np.random.permutation(len(b))
    return a[p], b[p]

In [11]:
def visualize_that_beh(history: dict):
    # summarize history for accuracy
    plt.plot(history["epoch"], history["cost"] , 'g', label='loss over epoch')
    plt.title('cost to epoch ')
    plt.ylabel('cost')
    plt.xlabel('epoch')
    plt.show()
    # summarize history for loss
    plt.plot(history["epoch"], history["learning_rate"], 'b', label='learning rate over epoch')
    plt.title('learning rate to epoch')
    plt.xlabel('Epochs')
    plt.ylabel('Learning Rate')
    plt.show()

In [12]:
def batch_iterator(parameters, labels, size_of_batch):
    l = np.expand_dims(a=labels, axis=-1)
    index = np.random.randint(0, len(parameters), (size_of_batch, parameters.shape[1])) # random sample
    mini_batch_x = np.take(parameters, index)
    mini_batch_y = np.take(labels, index)
    

    return mini_batch_x, mini_batch_y

In [13]:
def stochastic_gradient_descent(x, y, w, learning_rate: float, num_of_epochs: int, size_of_batch: int, size_of_dataset: int, reg_bool: bool = True):
    past_cost = 10000000
    current_cost = 0
    history = {
        "epoch": -1,
        "weight": -1,
        "bias": -1,
        "cost": -1,
        "batch_size": math.ceil(size_of_dataset/size_of_batch),
        "learning rate": -1
    }

    epochs_since_improved = 0

    for e in range(num_of_epochs - 1):
        shuffled_x, shuffled_y = unison_shuffled_copies(x, y)

        for b in range(math.ceil(size_of_dataset/size_of_batch) - 1):
            mini_batch_x, mini_batch_y = batch_iterator(x, y, size_of_batch)
            y_hat = model(mini_batch_x, w, b)

            if reg_bool == True:
                reg = l2(w, mini_batch_y)
                current_cost = regularized_mean_square_error(mini_batch_y, y_hat, reg)
            else: 
                current_cost = mean_square_error(y_hat, mini_batch_y)

            if (past_cost < current_cost): # if past cost remains to be lower than current cost, increment counter
                epochs_since_improved += 1

                if epochs_since_improved >= 3:
                    learning_rate /= 10 #learning rate decays if cost does not lower after 3 epochs 
                
            else: # if current cost is lower than past cost set the counter to zero and update the past cost value
                epoch_since_improved = 0 
                past_cost = current_cost

            w, b = gradient_descent(mini_batch_x, mini_batch_y, b, w, learning_rate, reg_bool)

            print("MSE: ", current_cost)

        history["epoch"] += 1
        history["weight"] = w
        history["bias"] = b
        history["cost"] = current_cost
        history["learning rate"] = learning_rate

    visualize_that_beh(history)
    
    return w, b
    


In [14]:
def find_lowest_loss(parameters, labels, learning_rate, num_of_epochs, size_of_batch, reg_bool = True):
    np.random.seed(0)
    w_shape = (parameters.shape[1], l.shape[1])
    scale = 1/max(1., (parameters.shape[1] + l.shape[1])/2.)

    size_of_dataset = len(labels)
    l = np.expand_dims(a=labels, axis=-1)
    w = () #set intial w value 
    b = np.zeros(l.shape[1]) #set initial bias value 
    reg = l2(w, labels) # set regularized term
    w_new, b_new = stochastic_gradient_descent(parameters, labels, w, learning_rate, num_of_epochs, size_of_batch, size_of_dataset, reg_bool)
    loss = train_valid_test(parameters, w, b, labels, reg, reg_bool)

In [15]:
def train_valid_test(x, w, b, y, reg: float, reg_bool: bool = False):
    y_hat = model(x, w, b)
    if reg_bool:
        train_or_validation = regularized_mean_square_error(y, y_hat, reg)
        test = regularized_mean_square_error(y, y_hat, reg)
    else:
        train_or_validation = mean_square_error(y_hat, y)
        test = mean_square_error(y_hat, y)

    return train_or_validation, test

In [16]:
def load_data():
    #training set
    x_tr = np.reshape(np.load("age_regression_Xtr.npy"), (-1, 48*48))
    y_tr = np.load("age_regression_ytr.npy")

    #testing set
    x_te = np.reshape(np.load("age_regression_Xte.npy"), (-1, 48*48))
    y_te = np.load("age_regression_yte.npy")

    return x_tr, y_tr, x_te, y_te

In [17]:
#Function Arguments:
#mse_bool is the boolean value that determines if mse will be performed without or with regularization, default is True
#alpha is the value for regularization, if used, default is 0.0
#ttv_val is the value associated with the type of split wanted. A Train/Test Split is 0, and a train/validation/split is 1, No Split Needed is 2. Default is 2
#learning rate is the hyperparameter associated with the gradient descent
def train_age_regressor(num_of_epochs: int, size_of_batch: int, ttv_val: int = 2, the_set: int = 0, learning_rate: float = 0.1):
    # Load data
    x_tr, y_tr, x_te, y_te = load_data()

    if ttv_val == 0:
        print("I made this before I realized train/test split is already given to you")

    elif ttv_val == 1:
        x_val, y_val, x_te, y_te = random_test_validation_split(x_te, y_te, train_perc = 0.8)

    loss = 1000000000
    if the_set == 0: #training set
        loss = find_lowest_loss(x_tr, y_tr, learning_rate, num_of_epochs, size_of_batch, False)
    elif the_set == 1: #validation set
        loss = find_lowest_loss(x_val, y_val, learning_rate, num_of_epochs, size_of_batch, False)
    elif the_set == 2: #test set
        loss = find_lowest_loss(x_te, y_te, learning_rate, num_of_epochs, size_of_batch, False)
    else:
        print("you did something wrong bud")


    return loss


    # Report fMSE cost on the training and testing data (separately)
    # ...

In [18]:
print(train_age_regressor(num_of_epochs=300,size_of_batch=100, ttv_val=1, the_set=0, learning_rate=0.1))

MSE:  2049563.2087066027
MSE:  7.975880252950522e+22
MSE:  3.5303907205798003e+39
MSE:  1.5635749000293804e+56
MSE:  6.935080030574707e+70
MSE:  3.078489195950324e+83
MSE:  1.3648993255958013e+94
MSE:  6.044563967050886e+102
MSE:  2.681025415789095e+109
MSE:  1.1809529851555533e+114
MSE:  4.7665526765554815e+116
MSE:  5.8476105882848335e+116
MSE:  3.649667937322716e+116
MSE:  3.501693263543795e+116
MSE:  3.490063088951129e+116
MSE:  3.484395772899047e+116
MSE:  3.486112022233724e+116
MSE:  3.482652353435932e+116
MSE:  3.487906483932186e+116
MSE:  3.4880584064573615e+116
MSE:  3.4780201867450385e+116
MSE:  3.487442033324012e+116
MSE:  3.4837788904161785e+116
MSE:  3.490396176154795e+116
MSE:  3.4827663788663536e+116
MSE:  3.4874301595804625e+116
MSE:  3.489290522397093e+116
MSE:  3.49001898905978e+116
MSE:  3.490484808630409e+116
MSE:  3.489810302212906e+116
MSE:  3.4757468504680626e+116
MSE:  3.484780884796922e+116
MSE:  3.487844351446385e+116
MSE:  3.482925331607204e+116
MSE:  3.48378

KeyboardInterrupt: 